**Load Libraries**

In [2]:
!pip install langchain langchain-openai openai  chromadb gradio dotenv  langchain-community
print("loaded successfully")

loaded successfully


In [3]:
!pip install --upgrade openai
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
openai_api_key=os.getenv("OPENAI_API_KEY")
openai_client=OpenAI(api_key=openai_api_key,base_url="https://openrouter.ai/api/v1")
print("first 5 chars ",{openai_api_key[:5]})

   ---------------------------------------- 0.0/948.4 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/948.4 kB ? eta -:--:--
   ---------------------- ----------------- 524.3/948.4 kB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 948.4/948.4 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.108.0
    Uninstalling openai-1.108.0:
      Successfully uninstalled openai-1.108.0
first 5 chars  {'sk-or'}


In [4]:
#load langchain components
from langchain_openai import OpenAIEmbeddings,OpenAI
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQAWithSourcesChain 

C:\Users\elwady\MiniConda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
DATA_FILE_PATH="eleven_madison_park_data.txt"
print(f"file path-> {DATA_FILE_PATH}")

file path-> eleven_madison_park_data.txt


In [6]:
loader=TextLoader(DATA_FILE_PATH,encoding="utf-8")
raw_document=loader.load()
print(f"loaded successfully {len(raw_document)} document")

loaded successfully 1 document


In [7]:
print(f"{raw_document[0].page_content[-750:] + "..." }")

, on October 1st, all of November will be made available. If you find that the reservation you were hoping for is booked, we do hold a waitlist and encourage you to add your name to it, and if something does become available, we will be in touch. Please contact careers@elevenmadisonpark.com or visit Culinary Agents . To stay up to date about future Eleven Madison Park news and events, please sign up for our Newsletter . Please contact events@elevenmadisonpark.com Please contact press@elevenmadisonpark.com Thank you for thinking of Eleven Madison Park for your inquiry. At this time, all of our non-profit efforts are focused on our partnership with Rethink Food . Email: info@elevenmadisonpark.com Phone: 212.889.0905 Ext. 3
---END OF SOURCE---...


In [8]:
#split the document into chunks
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=150)
documents=text_splitter.split_documents(raw_document)
if not documents:
    raise ValueError("Error: Splitting resulted in zero documents. Check the input file and splitter settings.")
print(f"document split int {len(documents)} chunks")

document split int 38 chunks


In [9]:
print("documet content..")
print(documents[0].page_content)
print("metadata...")
print(documents[0].metadata)      


documet content..
Source: https://www.elevenmadisonpark.com/
Title: Eleven Madison Park
Content:
Book on Resy
---END OF SOURCE---
metadata...
{'source': 'eleven_madison_park_data.txt'}


In [10]:
!pip install -U sentence-transformers langchain-huggingface

from langchain_community.embeddings import HuggingFaceEmbeddings

# pick one of the models above
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_store = Chroma.from_documents(documents=documents, embedding=embeddings)

print("Vector store created with", vector_store._collection.count(), "items")

  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 5.1.0
    Uninstalling sentence-transformers-5.1.0:
      Successfully uninstalled sentence-transformers-5.1.0


C:\Users\elwady\AppData\Local\Temp\ipykernel_20932\1902706906.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Vector store created with 38 items


In [11]:
stored_data = vector_store._collection.get(include=["embeddings", "documents"], limit = 1)  

# Display the results
print("First chunk text:\n", stored_data['documents'][0])
print("\nEmbedding vector:\n", stored_data['embeddings'][0])
print(f"\nFull embedding has {len(stored_data['embeddings'][0])} dimensions.")

First chunk text:
 Source: https://www.elevenmadisonpark.com/
Title: Eleven Madison Park
Content:
Book on Resy
---END OF SOURCE---

Embedding vector:
 [-2.90016439e-02 -6.03913404e-02  2.20067874e-02 -1.34712644e-02
  2.34959889e-02  7.93945193e-02  4.54231799e-02 -1.66562721e-02
 -3.57321054e-02  4.64659594e-02 -4.94946688e-02  9.23847482e-02
  1.55326352e-02 -1.20987622e-02 -4.72648852e-02  1.74570568e-02
  7.32015297e-02  7.89159983e-02 -1.59301031e-02 -2.90575717e-03
  8.87777209e-02  1.82440169e-02  2.59796213e-02 -1.75921638e-02
 -7.59837450e-03 -2.64163595e-02 -1.12898797e-01  4.02005278e-02
 -5.09870872e-02 -1.12363845e-02 -2.78260484e-02  7.42891431e-02
 -1.47451730e-02 -6.49156570e-02  6.97237924e-02  3.02265231e-02
  4.53479700e-02 -1.69542618e-02  7.08831921e-02  4.58162427e-02
 -8.35753828e-02 -1.16167823e-02 -3.51000018e-03  6.73603192e-02
 -6.62320703e-02  3.64346020e-02 -6.85378388e-02 -4.43773195e-02
  6.35948554e-02  2.74464227e-02  7.96251558e-03  3.75977978e-02
 -4.

In [11]:
test_query="Who is Daniel Humm?"
print(f"Searching for documents similar to: '{test_query}'")
try:
    similar_docs=vector_store.similarity_search(test_query,k=1)
    print(f"\nFound {len(similar_docs)} similar documents:")
    for i,doc in enumerate(similar_docs):
        print(f"\n--- Document {i+1} ---")
        content=doc.page_content[:700].strip()
        source=doc.metadata.get("source", "Unknown Source") 
        print(f"content:{content}")
        print(f"source:{source}")

except Exception as e:
    print(f"An error occurred during similarity search: {e}")

        
        


    

Searching for documents similar to: 'Who is Daniel Humm?'

Found 1 similar documents:

--- Document 1 ---
content:Source: https://www.elevenmadisonpark.com/press-and-accolades
Title: Press and Accolades — Eleven Madison Park
Content:
Accolades World’s 50 Best Best of the Best The New York Times Four Stars Michelin Guide Three Stars Wine Spectator Grand Award The World of Fine Wine Best Overall Wine List James Beard Foundation Outstanding Chef, Outstanding Service, Outstanding Pastry Chef, Outstanding Restaurant, Best Chef: NYC, Outstanding Wine Service, Rising Star Chef Recent Press Time: “Is Cooking for the 1% for a Reason” Washington Post: “The Joy of Plant-Based Eating” Interview: “Daniel Humm Is Begging You to Eat More Plants” Financial Times: "Epicurean escapes: Alain Ducasse and Daniel Humm’s tran
source:eleven_madison_park_data.txt


In [60]:
from langchain_openai import ChatOpenAI
retriever=vector_store.as_retriever(search_kwargs={"k":3})
llm=ChatOpenAI(temperature=0,openai_api_key=openai_api_key,
          openai_api_base="https://openrouter.ai/api/v1",
          model="mistralai/mistral-small-3.2-24b-instruct:free")
qa_chain=RetrievalQAWithSourcesChain.from_chain_type(llm=llm,
                                                    retriever=retriever,
                                                    chain_type="stuff",# chain_type="stuff": Puts all retrieved text directly into the prompt context.
#                      Suitable if the total text fits within the LLM's context limit.
                                                    return_source_documents = True,
                                                    verbose=True)


print("RetrievalQAWithSourcesChain created")

RetrievalQAWithSourcesChain created


In [12]:
print("\n Testing Full RAG Chain")
chain_test_query="what the name of chef"
try:
    result=qa_chain.invoke({"question":chain_test_query})
    print("answering")
    print(result.get("answer","No answer generated."))
    print("\n--- Sources ---")
    print(result.get("sources", "No sources identified."))

except Exception as e:
    print("there is an error")


 Testing Full RAG Chain
there is an error


In [13]:
def rag_query(user_query):
    try:
        result = qa_chain.invoke({"question": user_query})
        answer = result.get("answer", "No answer generated.")
        sources = result.get("sources", "No sources identified.")

        formatted_answer = f"### Answer\n{answer}\n\n### Sources\n{sources}"
        return formatted_answer
    except Exception as e:
        return f"⚠️ Error: {str(e)}"

# -----------------------------
# Gradio Interface
# -----------------------------
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 📚 RAG Q&A Assistant")
    with gr.Row():
        with gr.Column(scale=2):
            query_box = gr.Textbox(
                label="Ask a question:",
                placeholder="Type your question here...",
                lines=2,
            )
            submit_btn = gr.Button("🔍 Search & Answer")
        with gr.Column(scale=3):
            output_box = gr.Markdown(label="Answer")

    submit_btn.click(rag_query, inputs=query_box, outputs=output_box)

demo.launch()

NameError: name 'gr' is not defined